# Hyper Parameter Tunning (TF2)

하이퍼 파라미터 튜닝

https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=ko

In [1]:
# 설치
!pip install -q --upgrade pip
!pip install -q -U keras-tuner

In [2]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

# 노트북 셀 표시를 브라우저 전체 폭 사용하기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, shutil

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import tensorflow as tf
tf.random.set_seed(rseed)
tf.keras.backend.set_floatx('float32') # keras default float type 설정

import tensorflow_hub as tfhub
import tensorflow_datasets as tfds

import kerastuner as kt

import seaborn as sns

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")
print(f"tensorflow ver={tf.__version__}")
print(f"tensorflow execuring eagerly={tf.executing_eagerly()}")
print(f"tensorflow hub ver={tfhub.__version__}")
print(f"tensorflow GPU={'True' if tf.config.experimental.list_physical_devices('GPU') else 'False'}")
print(f"keras tuner ver={kt.__version__}")
print(f"senborn ver={sns.__version__}")

python ver=3.7.6 (default, Nov 21 2020, 22:51:13) 
[Clang 12.0.0 (clang-1200.0.32.27)]
pandas ver=1.0.5
numpy ver=1.19.5
tensorflow ver=2.4.1
tensorflow execuring eagerly=True
tensorflow hub ver=0.11.0
tensorflow GPU=False
keras tuner ver=1.0.2
senborn ver=0.10.0


## 소개

Keras Tuner는 TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트를 선택하는 데 도움을 주는 라이브러리입니다. 머신러닝(ML) 애플리케이션에 대한 올바른 하이퍼파라미터 세트를 선택하는 과정을 하이퍼파라미터 조정 또는 하이퍼튜닝이라고 합니다.

하이퍼파라미터는 훈련 프로세스 및 ML 모델의 토폴로지를 제어하는 변수입니다. 이러한 변수는 훈련 과정에서 일정하게 유지되며 ML 프로그램의 성능에 직접적으로 영향을 미칩니다. 하이퍼파라미터에는 두 가지 유형이 있습니다.

* 숨겨진 레이어의 수 및 너비와 같이 모델 선택에 영향을 미치는 모델 하이퍼파라미터 
* SGD(Stochastic Gradient Descent)의 학습률 및 KNN(k Nearest Neighbors) 분류자의 최근접 이웃 수와 같은 학습 알고리즘의 속도와 품질에 영향을 주는 알고리즘 하이퍼파라미터 

이 튜토리얼에서는 Keras Tuner를 사용하여 이미지 분류 애플리케이션에 하이퍼튜닝을 수행합니다.

## 데이터세트 다운로드 및 준비하기

이 튜토리얼에서는 Keras Tuner를 사용하여 [Fashion MNIST 데이터세트](https://github.com/zalandoresearch/fashion-mnist)에서 의류 이미지를 분류하는 머신러닝 모델에 가장 적합한 하이퍼파라미터를 찾습니다.

데이터를 로드합니다.

In [3]:
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 모델 정의하기

하이퍼튜닝을 위한 모델을 빌드할 때는 모델 아키텍처와 더불어 하이퍼파라미터 검색 공간도 정의합니다. 하이퍼튜닝을 위해 설정하는 모델을 하이퍼 모델이라고 합니다.

두 가지 접근 방식을 통해 하이퍼 모델을 정의할 수 있습니다.

* 모델 빌더 함수 사용
* Keras Tuner API의 HyperModel 클래스를 하위 클래스화

또한 두 개의 사전 정의된 HyperModel - 클래스인 HyperXception과 HyperResNet을 컴퓨터 비전 애플리케이션에 사용할 수 있습니다.

이 튜토리얼에서는 모델 빌더 함수를 사용하여 이미지 분류 모델을 정의합니다. 모델 빌더 함수는 컴파일된 모델을 반환하고 인라인으로 정의한 하이퍼파라미터를 사용하여 모델을 하이퍼튜닝합니다.

In [4]:
def model_builder(hp):
    model = tf.keras.Sequential()
    
    # 입력 레이어를 정의합니다.
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    # 첫번째 Dense Layer의 유닛 수를 32-512 사이의 값 중 최적의 값을 선택하여 튜닝합니다.
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(tf.keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(tf.keras.layers.Dense(10))

    # 옵티마이저를 위한 learning rate 값을 0.01, 0.001 또는 0.001 중 최적의 값을 선택하여 튜닝합니다. 
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

    return model

## 튜너를 인스턴스화하고 하이퍼튜닝 수행하기

튜너를 인스턴스화하여 하이퍼튜닝을 수행합니다. Keras Tuner에는 RandomSearch, Hyperband, BayesianOptimization 및 Sklearn의 네 가지 튜너가 있습니다. 이 튜토리얼에서는 Hyperband 튜너를 사용합니다.

Hyperband 튜너를 인스턴스화하려면 최적화할 하이퍼모델인 objective, 및 훈련할 최대 epoch 수(max_epochs)를 지정해야 합니다.

In [5]:
directory = '/tmp/my_dir'
if os.path.exists(directory) and os.path.isdir(directory):
    shutil.rmtree(directory)
    
tuner = kt.Hyperband(model_builder, objective = 'val_accuracy', 
                     max_epochs = 10, factor = 3, directory = directory, project_name = 'intro_to_kt')

Hyperband 튜닝 알고리즘은 적응형 리소스 할당 및 조기 중단을 사용하여 고성능 모델에서 신속하게 수렴합니다. 이것은 스포츠 챔피언십 스타일 브래킷을 사용하여 수행됩니다. 이 알고리즘은 몇 개의 epoch에 대해 많은 수의 모델을 훈련하고 최고 성능을 보이는 절반만 다음 단계로 넘깁니다. Hyperband는 1 + logfactor(max_epochs)를 계산하고 이를 가장 가까운 정수로 반올림하여 한 브래킷에서 훈련할 모델 수를 결정합니다.

하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의합니다.

In [6]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        clear_output(wait = True)

하이퍼파라미터 검색을 실행합니다. 검색 메서드의 인수는 위의 콜백 외에 tf.keras.model.fit에 사용되는 인수와 같습니다.

In [7]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
# https://keras-team.github.io/keras-tuner/documentation/tuners/#get_best_hyperparameters-method
# num_trials: (int, optional). Number of HyperParameters objects to return. HyperParameters will be returned in sorted order based on trial performance.
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.""")

Trial 30 Complete [00h 00m 14s]
val_accuracy: 0.8569999933242798

Best val_accuracy So Far: 0.8920999765396118
Total elapsed time: 00h 04m 09s
INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer is 0.001.


이 튜토리얼을 마치려면 검색에서 최적의 하이퍼파라미터로 모델을 재훈련합니다.

In [8]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5928 - accuracy: 0.7883 - val_loss: 0.3999 - val_accuracy: 0.8583
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3691 - accuracy: 0.8639 - val_loss: 0.4122 - val_accuracy: 0.8485
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3228 - accuracy: 0.8801 - val_loss: 0.3446 - val_accuracy: 0.8738
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3005 - accuracy: 0.8879 - val_loss: 0.3647 - val_accuracy: 0.8689
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2779 - accuracy: 0.8983 - val_loss: 0.3345 - val_accuracy: 0.8785
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2671 - accuracy: 0.8981 - val_loss: 0.3369 - val_accuracy: 0.8792
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2482 - accuracy: 0.9070 - val_loss: 0.3748 - val_accuracy:

/tmp/my_dir/intro_to_kt 디렉토리에는 하이퍼파라미터 검색 중에 실행되는 모든 시험(모델 구성)에 대한 상세 로그와 체크포인트가 들어 있습니다. 하이퍼파라미터 검색을 다시 실행하면 Keras Tuner가 이러한 로그의 기존 상태를 사용하여 검색을 재개합니다. 이 동작을 비활성화하려면 튜너를 인스턴스화하는 동안 추가 overwrite = True 인수를 전달합니다.

## 요약

이 튜토리얼에서는 Keras Tuner를 사용하여 모델의 하이퍼파라미터를 조정하는 방법을 배웠습니다. Keras Tuner에 대한 자세한 내용은 다음 추가 자료를 확인하세요.

[TensorFlow 블로그의 Keras Tuner](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html?hl=ko)
[Keras Tuner 웹 사이트](https://keras-team.github.io/keras-tuner/)

모델 하이퍼파라미터를 능동적으로 조정하기 위한 [TensorBoard의 HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams?hl=ko)도 확인해 보세요.

*****

Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 4.0 License](https://creativecommons.org/licenses/by/4.0/), and code samples are licensed under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). For details, see the [Google Developers Site Policies](https://developers.google.com/site-policies?hl=ko). Java is a registered trademark of Oracle and/or its affiliates.